In [20]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2

In [4]:
img = cv2.imread(r"C:\Users\supichik\Downloads\face-mask-dataset\Dataset\with_mask\4-with-mask.jpg")

In [5]:
print(img.shape)

(429, 311, 3)


In [6]:
import os

targets = ["with_mask","without_mask"]

path = r"C:\Users\supichik\Downloads\face-mask-dataset\Dataset"

a=0

data = []

for target in targets:
    fold_path = os.path.join(path,target)
    
    for image in os.listdir(fold_path):
        img_path = os.path.join(fold_path,image)
        img_array = cv2.imread(img_path)
        resized_img_array = cv2.resize(img_array,(224,224))
        label = targets.index(target)
        data.append([resized_img_array,label])
        


        

In [7]:
import random
random.shuffle(data)


# l = [[1,'a'],[2,'b']]

# fe1 = []
# la1 = []
# for fe,la in l:
#     fe1.append(fe)
#     la1.append(la)
# print(fe1)
# print(la1)

# o/p :
# [1, 2]
# ['a', 'b']


X = []
Y = []

for feature,label in data:
    X.append(feature)
    Y.append(label)


In [8]:
import numpy as np
X = np.array(X)
Y = np.array(Y)

In [9]:
X.shape

(1508, 224, 224, 3)

In [10]:
Y

array([0, 1, 1, ..., 0, 0, 0])

In [11]:
len(X)

1508

In [12]:
len(Y)

1508

In [13]:
#Standardizing the X

X = X/255

In [71]:
pip install scikit-learn

You should consider upgrading via the 'c:\users\supichik\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [14]:

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y)

In [15]:
X_train.shape

(1206, 224, 224, 3)

In [16]:
Y_train.shape

(1206,)

In [17]:
X_test.shape

(302, 224, 224, 3)

In [18]:
Y_test.shape

(302,)

In [84]:
pip install pandas


You should consider upgrading via the 'c:\users\supichik\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [19]:
#checking wether stratification is performed or not after train_test_split
import pandas as pd
r = Y_train
pd.Series(r).value_counts()

0    604
1    602
dtype: int64

In [20]:
m = Y_test
pd.Series(m).value_counts()

0    151
1    151
dtype: int64

In [21]:
l=[1,2,3,4,5]
print(l[:-1])

[1, 2, 3, 4]


In [94]:
pip install keras

You should consider upgrading via the 'c:\users\supichik\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [100]:
pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3



In [101]:
pip install tensorflow


  Using cached tensorflow-2.5.0rc2-cp39-cp39-win_amd64.whl (423.0 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached h5py-3.1.0-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached grpcio-1.34.1-cp39-cp39-win_amd64.whl (2.9 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached Markdown-3.3.4-py3-none-any.whl (97 kB)
  Using cached google_auth-1.30.0-py2.py3-none-any.whl (146 kB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
  Using cached google_auth_oauthlib-0.4.4-py2.py3-none-any.whl (18 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached tensorboard_data_server-0.6.0-py3-none-any.whl (2.3 kB)
  Using cached tensorboard_plugin_wit-1.8.0-py3-none-any.whl (781 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-

In [22]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [23]:
vgg=VGG16()

In [24]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [25]:
from keras import Sequential

In [26]:
model = Sequential()

In [27]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [29]:
for layer in model.layers:
    layer.trainable=False

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [31]:
from keras.layers import Dense

In [32]:
model.add(Dense(1,activation='sigmoid'))


In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [34]:
model.compile(optimizer="Adam",loss = "binary_crossentropy",metrics=['accuracy'])

In [35]:
model.fit(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Epoch 1/5
38/38 [==============================] - 490s 12s/step - loss: 0.6804 - accuracy: 0.5744 - val_loss: 0.3961 - val_accuracy: 0.8974
Epoch 2/5
38/38 [==============================] - 445s 12s/step - loss: 0.3601 - accuracy: 0.9008 - val_loss: 0.2990 - val_accuracy: 0.8940
Epoch 3/5
38/38 [==============================] - 478s 13s/step - loss: 0.2555 - accuracy: 0.9226 - val_loss: 0.2349 - val_accuracy: 0.9272
Epoch 4/5
38/38 [==============================] - 523s 14s/step - loss: 0.2281 - accuracy: 0.9302 - val_loss: 0.2019 - val_accuracy: 0.9404
Epoch 5/5
38/38 [==============================] - 509s 13s/step - loss: 0.1844 - accuracy: 0.9437 - val_loss: 0.2102 - val_accuracy: 0.9338


In [36]:
def prediction(imge,models):
    model=models
    y_pred = model.predict_classes(imge.reshape(1,224,224,3))
    return y_pred[0][0]

In [37]:
def draw_txt(image,text,org,font,fontscale,color,thickness):
    image = cv2.putText(image, text, org, font, fontscale, color, thickness)
    return image

In [38]:
def capturing_on_video(models):
    cap = cv2.VideoCapture(0)
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        resized_frame = cv2.resize(frame,(224,224))
        y_pred = prediction(resized_frame,models)
        if y_pred==0:
            image = draw_txt(frame,"MASK",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        else:
            image = draw_txt(frame,"NO_MASK",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        cv2.imshow("video",image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [43]:
pip install h5py


In [48]:
from keras.models import model_from_json

In [68]:
# serialize model to JSON/converting model to json object
model_json = model.to_json()
with open("vgg_last_layer.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [69]:
#loading model from json file
with open("vgg_last_layer.json", "r") as json_file:
    loaded_model_json = json_file.read()
vgg_last_lay_loaded_model = model_from_json(loaded_model_json)
# load weights into new model
vgg_last_lay_loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [43]:
#this helps us to print 0/1 as output based on the mask & no_mask on video
# def capturing_on_video(models):
#     cap = cv2.VideoCapture(0)
#     while True:
#         ret, frame = cap.read()
#         resized_frame = cv2.resize(frame,(224,224))
#         print(prediction(resized_frame,models))
#         cv2.imshow("video",frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()



In [74]:
capturing_on_video(vgg_last_lay_loaded_model)

In [52]:
img = cv2.imread(r"C:\Users\supichik\Downloads\face-mask-dataset\Dataset\with_mask\1-with-mask.jpg")

In [53]:
print(img)

[[[248 250 250]
  [248 250 250]
  [248 250 250]
  ...
  [249 251 251]
  [249 251 251]
  [249 251 251]]

 [[248 250 250]
  [248 250 250]
  [248 250 250]
  ...
  [249 251 251]
  [249 251 251]
  [249 251 251]]

 [[248 250 250]
  [248 250 250]
  [248 250 250]
  ...
  [249 251 251]
  [249 251 251]
  [249 251 251]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 20  22  22]
  [ 19  21  21]
  [ 18  20  20]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 20  22  22]
  [ 19  21  21]
  [ 18  20  20]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 20  22  22]
  [ 19  21  21]
  [ 18  20  20]]]


In [54]:
img = img/255


In [55]:
resimg = cv2.resize(img,(224,224))

In [56]:
resimg.shape

(224, 224, 3)

In [57]:
print(prediction(resimg,vgg_last_lay_loaded_model))

0


In [58]:
#Building another VGG16 model with last 3 dense layers trianable
model_1 = Sequential()

In [59]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [60]:
for layer in vgg.layers[:-3]:
    model_1.add(layer)


In [61]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [62]:
for layer in model_1.layers:
    layer.trainable=False


In [63]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [64]:
model_1.add(Dense(512, activation='relu'))
model_1.add(Dense(512,activation='relu'))
model_1.add(Dense(1,activation='sigmoid'))
model_1.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [65]:
model_1.compile(optimizer="Adam",loss = "binary_crossentropy",metrics=['accuracy'])

In [66]:
model_1.fit(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Epoch 1/5
38/38 [==============================] - 472s 12s/step - loss: 2.0671 - accuracy: 0.6436 - val_loss: 0.0363 - val_accuracy: 0.9834
Epoch 2/5
38/38 [==============================] - 441s 12s/step - loss: 0.0230 - accuracy: 0.9913 - val_loss: 0.0260 - val_accuracy: 0.9901
Epoch 3/5
38/38 [==============================] - 433s 11s/step - loss: 0.0116 - accuracy: 0.9967 - val_loss: 0.0457 - val_accuracy: 0.9801
Epoch 4/5
38/38 [==============================] - 426s 11s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0335 - val_accuracy: 0.9868
Epoch 5/5
38/38 [==============================] - 460s 12s/step - loss: 2.9378e-04 - accuracy: 1.0000 - val_loss: 0.0311 - val_accuracy: 0.9868


In [71]:
# serialize model to JSON/converting model to json object
model_json = model_1.to_json()
with open("vgg_last_3_layer.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_1.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [72]:
#loading model from json file
with open("vgg_last_3_layer.json", "r") as json_file:
    loaded_model_json = json_file.read()
vgg_last_3_lay_loaded_model = model_from_json(loaded_model_json)
# load weights into new model
vgg_last_3_lay_loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [75]:
capturing_on_video(vgg_last_3_lay_loaded_model)